# Exploring with Spark and Pandas

### Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

In [2]:
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [3]:
# Joining the data frames:
df = case.join(dept, 'dept_division', 'left').join(source, 'source_id', 'left')

In [6]:
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 1/1/18 0:42                          
 case_closed_date       | 1/1/18 12:29                         
 SLA_due_date           | 9/26/20 0:42                         
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

#### 1. How many different cases are there, by department?

In [11]:
df.select(df.case_id).groupBy(df.dept_division).count()

AnalysisException: Resolved attribute(s) dept_division#23 missing from case_id#16 in operator !Aggregate [dept_division#23], [dept_division#23, count(1) AS count#323L].;
!Aggregate [dept_division#23], [dept_division#23, count(1) AS count#323L]
+- Project [case_id#16]
   +- Project [source_id#27, dept_division#23, case_id#16, case_opened_date#17, case_closed_date#18, SLA_due_date#19, case_late#20, num_days_late#21, case_closed#22, service_request_type#24, SLA_days#25, case_status#26, request_address#28, council_district#29, dept_name#61, standardized_dept_name#62, dept_subject_to_SLA#63, source_username#85]
      +- Join LeftOuter, (source_id#27 = source_id#84)
         :- Project [dept_division#23, case_id#16, case_opened_date#17, case_closed_date#18, SLA_due_date#19, case_late#20, num_days_late#21, case_closed#22, service_request_type#24, SLA_days#25, case_status#26, source_id#27, request_address#28, council_district#29, dept_name#61, standardized_dept_name#62, dept_subject_to_SLA#63]

#### 2. Does the percentage of cases that are late vary by department?

#### 3. On average, how late are the late cases by department?

#### 4. What is the service type that is the most late? Just for Parks & Rec?

#### 5. For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.

#### 6. Does whether or not its a weekend matter for when a case is opened/closed?

#### 7. On average, how many cases are opened a day for the Customer Service department?

#### 8. Does the number of service requests for the solid waste department vary by day of the week?